# Part (a)

i. 8 decision variables, one for each type of stock. They represent the amount of shares we decide to sell for each stock.

ii. Maximum values are the amount of each share we own. The minimum value is zero, where we do not sell anything.

iii. $34.55(150-x)$

iv. $50*23.67 - 0.01(50*23.67) - 0.3*50*(23.67-22.54) = 1154.715$

# Part (b)

In [1]:
# LOAD PACKAGES
# -------------
using JuMP
using DataFrames
using CSV
using GLPK

In [2]:
# INPUT DATA
# ----------
n = 1:8
Inv = DataFrame!(CSV.File("Investment.csv")) #all coefficients are stored here

,Number,Stock,Number of Shares,Price Purchased Last Year,Current Price
,Int64,String,Int64,Float64,Float64
1,1,Yahoo!,150,15.68,31.8
2,2,General Electric,150,22.1,24.28
3,3,Microsoft,150,30.39,32.5
4,4,Bank of America,150,8.93,14.16
5,5,JPMorgan Chase,150,40.55,50.99
6,6,"Cisco Systems, Inc",150,18.58,24.17
7,7,Intel,150,22.54,23.67
8,8,Pfizer,150,24.84,28.77


In [3]:
# MODEL CONSTRUCTION
# ------------------
model = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [4]:
# VARIABLES
# ---------
@variable(model, 0 <= x[n] <= 150)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, 1:8
And data, a 8-element Array{VariableRef,1}:
 x[1]
 x[2]
 x[3]
 x[4]
 x[5]
 x[6]
 x[7]
 x[8]

In [5]:
# CONSTRAINTS
# -----------
@constraint(model, sum((x[i]*Inv[i,5])-(0.01*x[i]*Inv[i,5])-(0.3*(x[i]*(Inv[i,5]-Inv[i,4]))) for i = n)>= 10000)

26.646 x[1] + 23.383200000000002 x[2] + 31.541999999999994 x[3] + 12.449399999999999 x[4] + 47.348099999999995 x[5] + 22.251299999999997 x[6] + 23.094300000000004 x[7] + 27.3033 x[8] >= 10000.0

In [6]:
# OBJECTIVE
# ---------
@objective(model, Max, sum(Inv[i,6]*(150-x[i]) for i = n))

-29.5 x[1] - 26.31 x[2] - 34.55 x[3] - 15.23 x[4] - 62.43 x[5] - 26.68 x[6] - 23.85 x[7] - 31.66 x[8] + 37531.5

In [7]:
# CHECK THE MODEL
# --------
println("The optimization problem to be solved is:")
print(model)

The optimization problem to be solved is:
Max -29.5 x[1] - 26.31 x[2] - 34.55 x[3] - 15.23 x[4] - 62.43 x[5] - 26.68 x[6] - 23.85 x[7] - 31.66 x[8] + 37531.5
Subject to
 26.646 x[1] + 23.383200000000002 x[2] + 31.541999999999994 x[3] + 12.449399999999999 x[4] + 47.348099999999995 x[5] + 22.251299999999997 x[6] + 23.094300000000004 x[7] + 27.3033 x[8] >= 10000.0
 x[1] >= 0.0
 x[2] >= 0.0
 x[3] >= 0.0
 x[4] >= 0.0
 x[5] >= 0.0
 x[6] >= 0.0
 x[7] >= 0.0
 x[8] >= 0.0
 x[1] <= 150.0
 x[2] <= 150.0
 x[3] <= 150.0
 x[4] <= 150.0
 x[5] <= 150.0
 x[6] <= 150.0
 x[7] <= 150.0
 x[8] <= 150.0


In [8]:
# SOLVE IT
# --------
status = optimize!(model) # solves the model  
termination_status(model)

OPTIMAL::TerminationStatusCode = 1

In [9]:
# DISPLAY THE RESULTS
# -------------------
println("i.")
println("Objective value: ", objective_value(model)) 
for i in n
    println(Inv[i,2], ": ", value.(x[i]))
end


i.
Objective value: 26773.662707348194
Yahoo!: 67.72329805599341
General Electric: 0.0
Microsoft: 150.0
Bank of America: 0.0
JPMorgan Chase: 0.0
Cisco Systems, Inc: 0.0
Intel: 150.0
Pfizer: 0.0


ii. If we want a diverse portfolio, we are not happy with this solution as we give up all our shares in 2 stocks.

# Part (c)

In [10]:
@constraint(model, Ub[i = n], x[i] <= 75)
status = optimize!(model) # solves the model  
termination_status(model)
println("Objective value: ", objective_value(model)) 
#println("x = ", value.(x)) 
for i in n
    println(Inv[i,2], ": ", value.(x[i]))
end

Objective value: 26468.541160516463
Yahoo!: 75.00000000000004
General Electric: 75.0
Microsoft: 75.00000000000001
Bank of America: 0.0
JPMorgan Chase: 0.0
Cisco Systems, Inc: 4.599281839712725
Intel: 74.99999999999997
Pfizer: 75.0
